In [5]:
import numpy as np
import scanpy as sc
from fbpca import pca
import time
from geosketch import gs
import pandas as pd
import glob
import os


### We are creating sketches of each dataset to process with gene panel selection tools

In [ ]:
path_to_loom_files = '/storage/adult_brain_genes/loom_files'
path_to_linnarson_files = '/storage/adult_brain_genes'

In [ ]:
fls = glob.glob(os.path.join(path_to_loom_files, '*.loom'))

In [ ]:
for fl in fls:
    print(fl)
    adata = sc.read_loom(fl)
    #preprocess
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    sc.pp.filter_cells(adata, min_genes=500)
    sc.pp.filter_genes(adata, min_cells=5)
    # qc
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    adata = adata[adata.obs.n_genes_by_counts < 4000, :]
    adata = adata[adata.obs.pct_counts_mt < 5, :]
    #more preprocess
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata)
    adata.raw = adata

    #get the superclusters from the cluster annotation file
    cluster_annotations = pd.read_csv(os.path.join(path_to_linnarson_files, 'cluster_annotation.csv'), delimiter='@')
    cluster_main = cluster_annotations['Supercluster'].tolist()

    #need to remove some parts of the following code to clean it up. Adding supercluster to adata
    cluster_true = []
    for i in cluster_main:
        cluster_true.append(i)
    cluster_annotations['cluster_true'] = cluster_true 
    enriched_genes = cluster_annotations['Top Enriched Genes'].tolist()

    gene_dictionary = {}
    for i in cluster_true:
        gene_dictionary[i] = []
    lin_genes = []
    for i in range(len(enriched_genes)):
        try:
            for j in enriched_genes[i].split(','):
                gene_dictionary[cluster_true[i]].append(j)
        except:
            print('float')

    lin_genes= []
    for key in gene_dictionary:
        sb = gene_dictionary.get(key)
        for k in sb:
            lin_genes.append(k.strip())
    lin_genes=list(set(lin_genes))
    unique_clusters = np.unique(adata.obs['Clusters'].tolist(), return_counts=True)[1]
    general_groups = []
    for cl in adata.obs['Clusters']:
        general_groups.append(cluster_annotations.loc[cl]['cluster_true'])
    adata.obs['celltype'] = pd.Categorical(general_groups)

    X_matrix = np.array(adata[:,adata.var.highly_variable].X.todense())

    N = 10000
    U, s, Vt = pca(X_matrix[:,:], k=100) # E.g., 100 PCs.
    X_dimred = U[:, :100] * s[:100]


    time1 = time.time()

    sketch_index = gs(X_dimred, N, replace=False)

    X_sketch = X_dimred[sketch_index]
    print(time.time()- time1)

    adata_sketch = adata[sketch_index, :]

    sc.tl.pca(adata_sketch)

    sc.pp.neighbors(adata_sketch)
    sc.tl.umap(adata_sketch)

    adata_sketch.write(os.path.join(path_to_linnarson_files,'sketches', os.path.basename(fl).split('.')[0]+'.h5ad'))

/storage/adult_brain_genes/loom_files/CerebralCortex.loom
